# Setup

In [ ]:
# 3rd party library imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import t as tdist
import seaborn as sns
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm

sns.set()
plt.rcParams['figure.figsize'] = [8.0, 4.8]

# Data Preparation

In [ ]:
df = pd.read_csv('case1301.csv')

# add a column for the ordinal average response by block
# use this to make a plot that seems to uniformly rise left to right
dfb = df.groupby('Block').mean(numeric_only=True).sort_values(by='Cover').reset_index()

df['BlockOrd'] = 0
for idx, row in dfb.iterrows():
    df.loc[df['Block'] == row['Block'], 'BlockOrd'] = idx + 1

# Analysis of the seaweed grazer data

## Initial assessment of additivity, outliers, and the need for transformation

In [ ]:
fig, ax = plt.subplots()
data = df.groupby(['Block', 'Treat']).mean().reset_index().sort_values(by='BlockOrd')
g = sns.lineplot(data=data, x='Block', y='Cover', hue='Treat', sort=False)
ax.set_xlabel('Block Number (ordered from smallest to largest average response)')
ax.set_ylabel('Percentage Seaweed Regrowth')

title = (
    'Average Percentages of seaweed regeneration '
    'with different grazers allowed'
)
ax.set_title(title);


In [ ]:
formula = 'Cover ~ Block * Treat'
sat_model_percent = smf.ols(formula=formula, data=df).fit()

fig, ax = plt.subplots()
g = sns.scatterplot(data=df, x='Cover', y=sat_model_percent.resid)
ax.set_xlabel('Fitted Percent Regeneration')
ax.set_ylabel('Residuals')
ax.set_title('Residual plot from the saturated model fit to the seaweed grazer data');

# Transformation

In [ ]:
df['rr'] = np.log(df['Cover'] / (100 - df['Cover']))

In [ ]:
fig, ax = plt.subplots()
data = df.groupby(['Block', 'Treat']).mean().reset_index().sort_values(by='BlockOrd')
g = sns.lineplot(data=data, x='Block', y='rr', hue='Treat', sort=False)
ax.set_xlabel('Block Number (ordered from smallest to largest average response)')
ax.set_ylabel('Regeneration Ratio Seaweed Regrowth');


### The analysis of variance table from the fit to the saturated model


In [ ]:
formula = 'rr ~ Block * Treat'                                                     
sat_model = smf.ols(formula=formula, data=df).fit()                                
sat_table = anova_lm(sat_model)                                                    
print(sat_table)   

In [ ]:
add_model = smf.ols(formula='rr ~ Block + Treat', data=df).fit()                                
add_table = anova_lm(add_model) 
print(add_table)

In [ ]:
anova_lm(add_model, sat_model)

We consider the interaction term in the saturated model

$
H_0: \mu\{Y|BLOCK, TREAT\} = BLOCK + TREAT
$

$
H_a: \mu\{Y|BLOCK,TREAT\} = BLOCK + TREAT + (BLOCK \times TREAT)
$

We conclude there is weak evidence for the interaction term 
($F_{35,48} = \frac{\frac{29.77 - 14.54}{83 - 48}}{14.54 / 48} = 1.4369$, $p = 0.1209$).

We consider the treatment effect in the additive model.

$
H_0: \mu\{Y|BLOCK,TREAT\} = BLOCK + TREAT
$

$
H_a: \mu\{Y|BLOCK, TREAT\} = BLOCK
$

In [ ]:
formula = 'rr ~ Block'                                                     
block_model = smf.ols(formula=formula, data=df).fit()                                
block_table = anova_lm(block_model)                                                    
print(block_table)
anova_lm(block_model, add_model)

We conclude there is strong evidence for the treatment effect ($F_{5,83} = 54.09$, $p < 0.0001$)

## Answers to specific questions of interest using linear combinations

### Table of averages of log percentage of seaweed regeneration ratio with different grazer combinations in eight blocks

In [ ]:
pd.options.display.float_format = '{:.2f}'.format
dfp = (
    df.groupby(['Block', 'Treat'])['rr']
      .mean()
      .reset_index()
      .pivot_table(index='Block', columns='Treat', values='rr', margins=True, margins_name='average')
)

# subtract off the overall mean from the block/treat column/row to get the block/treat effects
dfp['block effect'] = dfp['average'] - df['rr'].mean()
dfp.loc['treat effect', :] = dfp.loc['average', :] - df['rr'].mean()
dfp

### Do large fish have an effect on the regeneration ratio?
The difference between means from $fF$ and $f$ treatments measures this effect in the presence of small fish only; the difference between means from the $LfF$ and $Lf$ treatments measures the effect in the presence of both small fish and limpets.  The large fish effect is taken to be the average of those two effects:  $\gamma_1 = \frac{1}{2}(\mu_{fF} - \mu{f}) + \frac{1}{2}(\mu_{LfF} - \mu_{Lf})$.  This effect averages over different limpet conditions, so it measures a meaningful effect only if there is no limpet-by-big-fish interaction.

### Do small fish have an effect on the regeneration ratio?
This is investigated through the average of the difference between the $f$ and the $C$ treatment means and the $Lf$ and $L$ treatment means:  $\gamma_2 = \frac{1}{2} (\mu_f - \mu_C) + \frac{1}{2}(\mu_{Lf} - \mu_{L})$.

### Do limpets have an effect on the regeneration ratio?
This is investigated through $\gamma_3 = \frac{1}{3}(\mu_L - \mu_C) + \frac{1}{3}(\mu_{Lf} - \mu_{f}) + \frac{1}{3} (\mu_{LfF} - \mu_{fF})$

### Do limpets have a different effect when small fish are present than when small fish are not present?
When small fish are present, the limpet effect is given by $\frac{1}{2} (\mu_{Lf} - \mu_{f}) + \frac{1}{2} (\mu_{LfF} - \mu_{fF})$.  When small fish are not present, the limpet effect is $(\mu_{L} - \mu_{C})$.  The difference in effects is then $\gamma_4 = \frac{1}{2} (\mu_{Lf} - \mu_{f}) + \frac{1}{2} (\mu_{LfF} - \mu_{fF}) - (\mu_{L} - \mu_{C})$.

### Do limpets have a different effect when large fish are present than when large fish are absent?
This is investigated through $(\mu_{LfF} - \mu_{fF}) - (\mu_{Lf} - \mu_f)$.

$SE(g) = s_p \sqrt{\sum_{i = 1}^{I} \frac{C_{i}^2}{n_i}}$

In [ ]:
sp = np.sqrt(add_table.loc['Residual', 'mean_sq'])
I = len(df['Treat'].unique())
dof_sp = len(df) - I
s1 = 1 / 2 * pd.Series(index=['fF', 'f', 'LfF', 'Lf'], data=np.array([1,-1,1,-1]))
s2 = 1/2 * pd.Series(index=['f', 'CONTROL', 'Lf', 'L'], data=np.array([1,-1,1,-1]))
s3 = 1/3 * pd.Series(index=['L', 'CONTROL', 'Lf', 'f', 'LfF', 'fF'], data = [1,-1,1,-1,1,-1])
s4 = pd.Series(index=['Lf', 'f', 'LfF', 'fF', 'L', 'CONTROL'], data=[1/2,-1/2,1/2,-1/2,-1,1])
s5 = pd.Series(index=['LfF', 'fF', 'Lf', 'f'], data=[1,-1,-1,1])
dfC = pd.DataFrame([s1, s2, s3, s4, s5])
dfC = dfC.fillna(0)
dfn = 1 / df.groupby('Treat').size()
dfC, dfn

In [ ]:
se_g = pd.Series(sp * np.sqrt(dfC ** 2 @ dfn))
se_g

In [ ]:
estimate = dfC @ df.groupby('Treat').mean(numeric_only=True)['rr']
tstat = estimate / se_g
tstat

In [ ]:
pvalue = tdist.cdf(tstat, dof_sp) * 2
pvalue